In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

<ipython-input-1-001cdbaf02bd>:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)  # or 199


In [2]:
df_orig = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0120ENv3/Dataset/ML0101EN_EDX_skill_up/cbb.csv')

## Data Preparation - Cleaning and Replacement

In [3]:
df_orig.head()

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,North Carolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,18.2,40.7,30.0,32.3,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
1,Villanova,BE,40,35,123.1,90.9,0.9703,56.1,46.7,16.3,20.6,28.2,29.4,34.1,30.0,57.4,44.1,36.2,33.9,66.7,8.9,Champions,2.0,2016
2,Notre Dame,ACC,36,24,118.3,103.3,0.8269,54.0,49.5,15.3,14.8,32.7,32.1,32.9,26.0,52.9,46.5,37.4,36.9,65.5,2.3,E8,6.0,2016
3,Virginia,ACC,37,29,119.9,91.0,0.9600,54.8,48.4,15.1,18.8,29.9,25.2,32.1,33.4,52.6,46.3,40.3,34.7,61.9,8.6,E8,1.0,2016
4,Kansas,B12,37,32,120.9,90.4,0.9662,55.7,45.1,17.8,18.5,32.2,27.9,38.6,37.3,52.7,43.4,41.3,32.5,70.1,11.6,E8,1.0,2016


In [4]:
df = df_orig.copy()
df.POSTSEASON.fillna("OTHER",inplace = True)
df.POSTSEASON.value_counts()

OTHER        1134
R64          128 
R32          64  
S16          32  
E8           16  
R68          16  
F4           8   
2ND          4   
Champions    4   
Name: POSTSEASON, dtype: int64

In [5]:
position_list = df.POSTSEASON.unique().tolist()
position_map = {'2ND':1, 'Champions':1, 'E8':1, 'F4':1, 'OTHER':0, 'R32':0, 'R64':0, 'R68':0, 'S16': 0}
df.POSTSEASON = df.POSTSEASON.map(position_map)
df.isnull().sum()

TEAM          0   
CONF          0   
G             0   
W             0   
ADJOE         0   
ADJDE         0   
BARTHAG       0   
EFG_O         0   
EFG_D         0   
TOR           0   
TORD          0   
ORB           0   
DRB           0   
FTR           0   
FTRD          0   
2P_O          0   
2P_D          0   
3P_O          0   
3P_D          0   
ADJ_T         0   
WAB           0   
POSTSEASON    0   
SEED          1134
YEAR          0   
dtype: int64

In [6]:
df["WINPERCENT"] = (df["W"]/df["G"])*100
df.drop(["SEED"],inplace = True,axis = 1) # As 1134 values are missing

In [7]:
df["TEAM"].replace({"IPFW": "Fort Wayne", "Arkansas Little Rock": "Little Rock"}, inplace=True)

In [8]:
year_list = df.YEAR.unique().tolist()

## H1 Feature Engineering

#### We have data for 2016, 2017, 2018, 2019.
#### Training Data - 2016 and 2017 with label mentioned below
#### Test Data - 2017 and 2018 with label mentioned below

### Label data - 
#### 2018 POSTSEASON will be used as label for 16-17 training data
#### 2019 POSTSEASON will be used as label for 17-18 test data

In [9]:
##########################
#Training_data Preparation
##########################

llast_year = year_list[0]
last_year = year_list[1]

data_llast = df[df.YEAR == llast_year]
data_llast.drop("YEAR",inplace = True, axis = 1)
col_list = data_llast.columns.tolist()
data_llast.columns = [str(col) + '_'+ 'llast' if col != "TEAM" else str(col) for col in data_llast.columns]

data_last = df[df.YEAR == last_year]
data_last.drop(["YEAR"],inplace = True, axis = 1)
data_last.columns = [str(col) + '_'+ 'last'  if col != "TEAM" else str(col)  for col in data_last.columns]

training_data = pd.merge(data_last, data_llast, on="TEAM")

for col in col_list[2:]:
    training_data[col + "_delta"] = (training_data[col + "_" + 'last'] - training_data[col + "_" + 'llast'])/training_data[col + "_" + 'last']

for col in col_list[2:]:
    training_data[col + "_diff"] = (training_data[col + "_" + 'last'] - training_data[col + "_" + 'llast'])
    
training_data["win_count"] = training_data["POSTSEASON_llast"]+ training_data["POSTSEASON_last"]

training_data.drop(["POSTSEASON_delta"],axis = 1,inplace = True)
training_data.drop(["POSTSEASON_diff"],axis = 1,inplace = True)

training_data.isnull().sum()

C:\Users\shuba\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\shuba\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


TEAM                0
CONF_last           0
G_last              0
W_last              0
ADJOE_last          0
ADJDE_last          0
BARTHAG_last        0
EFG_O_last          0
EFG_D_last          0
TOR_last            0
TORD_last           0
ORB_last            0
DRB_last            0
FTR_last            0
FTRD_last           0
2P_O_last           0
2P_D_last           0
3P_O_last           0
3P_D_last           0
ADJ_T_last          0
WAB_last            0
POSTSEASON_last     0
WINPERCENT_last     0
CONF_llast          0
G_llast             0
W_llast             0
ADJOE_llast         0
ADJDE_llast         0
BARTHAG_llast       0
EFG_O_llast         0
EFG_D_llast         0
TOR_llast           0
TORD_llast          0
ORB_llast           0
DRB_llast           0
FTR_llast           0
FTRD_llast          0
2P_O_llast          0
2P_D_llast          0
3P_O_llast          0
3P_D_llast          0
ADJ_T_llast         0
WAB_llast           0
POSTSEASON_llast    0
WINPERCENT_llast    0
G_delta   

In [10]:
tlabel_year = year_list[2]
train_label_data = df[df.YEAR == tlabel_year]
train_label_data = train_label_data[["TEAM","POSTSEASON"]]
training_data = pd.merge(training_data, train_label_data, on="TEAM")

In [11]:
##########################
#Test_data Preparation
##########################
llast_year = year_list[1]
last_year = year_list[2]

data_llast = df[df.YEAR == llast_year]
data_llast.drop("YEAR",inplace = True, axis = 1)
col_list = data_llast.columns.tolist()
data_llast.columns = [str(col) + '_'+ 'llast' if col != "TEAM" else str(col) for col in data_llast.columns]

data_last = df[df.YEAR == last_year]
data_last.drop(["YEAR"],inplace = True, axis = 1)
data_last.columns = [str(col) + '_'+ 'last'  if col != "TEAM" else str(col)  for col in data_last.columns]

test_data = pd.merge(data_last, data_llast, on="TEAM")

for col in col_list[2:]:
    test_data[col + "_delta"] = (test_data[col + "_" + 'last'] - test_data[col + "_" + 'llast'])/test_data[col + "_" + 'last']

for col in col_list[2:]:
    test_data[col + "_diff"] = (test_data[col + "_" + 'last'] - test_data[col + "_" + 'llast'])

test_data["win_count"] = test_data["POSTSEASON_llast"] + test_data["POSTSEASON_last"]

    
test_data.drop(["POSTSEASON_delta"],axis = 1,inplace = True)
test_data.drop(["POSTSEASON_diff"],axis = 1,inplace = True)

test_data.isnull().sum()

C:\Users\shuba\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\shuba\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


TEAM                0
CONF_last           0
G_last              0
W_last              0
ADJOE_last          0
ADJDE_last          0
BARTHAG_last        0
EFG_O_last          0
EFG_D_last          0
TOR_last            0
TORD_last           0
ORB_last            0
DRB_last            0
FTR_last            0
FTRD_last           0
2P_O_last           0
2P_D_last           0
3P_O_last           0
3P_D_last           0
ADJ_T_last          0
WAB_last            0
POSTSEASON_last     0
WINPERCENT_last     0
CONF_llast          0
G_llast             0
W_llast             0
ADJOE_llast         0
ADJDE_llast         0
BARTHAG_llast       0
EFG_O_llast         0
EFG_D_llast         0
TOR_llast           0
TORD_llast          0
ORB_llast           0
DRB_llast           0
FTR_llast           0
FTRD_llast          0
2P_O_llast          0
2P_D_llast          0
3P_O_llast          0
3P_D_llast          0
ADJ_T_llast         0
WAB_llast           0
POSTSEASON_llast    0
WINPERCENT_llast    0
G_delta   

In [12]:
telabel_year = year_list[3]
test_label_data = df[df.YEAR == telabel_year]
test_label_data = test_label_data[["TEAM","POSTSEASON"]]
test_data = pd.merge(test_data, test_label_data, on="TEAM")

In [13]:
test_data.shape, training_data.shape

((351, 87), (351, 87))

In [14]:
training_data.head()

,TEAM,CONF_last,G_last,W_last,ADJOE_last,ADJDE_last,BARTHAG_last,EFG_O_last,EFG_D_last,TOR_last,TORD_last,ORB_last,DRB_last,FTR_last,FTRD_last,2P_O_last,2P_D_last,3P_O_last,3P_D_last,ADJ_T_last,WAB_last,POSTSEASON_last,WINPERCENT_last,CONF_llast,G_llast,W_llast,ADJOE_llast,ADJDE_llast,BARTHAG_llast,EFG_O_llast,EFG_D_llast,TOR_llast,TORD_llast,ORB_llast,DRB_llast,FTR_llast,FTRD_llast,2P_O_llast,2P_D_llast,3P_O_llast,3P_D_llast,ADJ_T_llast,WAB_llast,POSTSEASON_llast,WINPERCENT_llast,G_delta,W_delta,ADJOE_delta,ADJDE_delta,BARTHAG_delta,EFG_O_delta,EFG_D_delta,TOR_delta,TORD_delta,ORB_delta,DRB_delta,FTR_delta,FTRD_delta,2P_O_delta,2P_D_delta,3P_O_delta,3P_D_delta,ADJ_T_delta,WAB_delta,WINPERCENT_delta,G_diff,W_diff,ADJOE_diff,ADJDE_diff,BARTHAG_diff,EFG_O_diff,EFG_D_diff,TOR_diff,TORD_diff,ORB_diff,DRB_diff,FTR_diff,FTRD_diff,2P_O_diff,2P_D_diff,3P_O_diff,3P_D_diff,ADJ_T_diff,WAB_diff,WINPERCENT_diff,win_count,POSTSEASON
0,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,17.1,30.0,26.2,39.0,26.9,56.3,40.0,38.2,29.0,71.5,7.7,1,94.871795,WCC,35,27,117.4,94.5,0.9238,55.2,44.8,17.1,15.1,32.1,26.0,34.4,28.1,54.3,44.4,37.8,30.3,68.2,2.1,0,77.142857,0.102564,0.270270,0.003396,-0.095017,0.050370,0.024735,-0.090024,-0.055556,0.116959,-0.070000,0.007634,0.117949,-0.044610,0.035524,-0.110000,0.010471,-0.044828,0.046154,0.727273,0.186873,4,10,0.4,-8.2,0.0490,1.4,-3.7,-0.9,2.0,-2.1,0.2,4.6,-1.2,2.0,-4.4,0.4,-1.3,3.3,5.6,17.728938,1,0
1,North Carolina,ACC,39,33,121.0,91.5,0.9615,51.7,48.1,16.2,18.6,41.3,25.0,34.3,31.6,51.0,46.3,35.5,33.9,72.8,8.4,1,84.615385,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,18.2,40.7,30.0,32.3,30.4,53.9,44.6,32.7,36.2,71.7,8.6,1,82.500000,-0.025641,0.000000,-0.019008,-0.037158,0.008736,-0.017408,0.000000,0.049383,0.021505,0.014528,-0.200000,0.058309,0.037975,-0.056863,0.036717,0.078873,-0.067847,0.015110,-0.023810,0.025000,-1,0,-2.3,-3.4,0.0084,-0.9,0.0,0.8,0.4,0.6,-5.0,2.0,1.2,-2.9,1.7,2.8,-2.3,1.1,-0.2,2.115385,2,0
2,Kansas,B12,36,31,121.5,94.5,0.9472,56.1,48.1,17.6,18.6,34.1,29.7,36.0,30.0,53.6,45.3,40.4,35.6,71.4,11.0,1,86.111111,B12,37,32,120.9,90.4,0.9662,55.7,45.1,17.8,18.5,32.2,27.9,38.6,37.3,52.7,43.4,41.3,32.5,70.1,11.6,1,86.486486,-0.027778,-0.032258,0.004938,0.043386,-0.020059,0.007130,0.062370,-0.011364,0.005376,0.055718,0.060606,-0.072222,-0.243333,0.016791,0.041943,-0.022277,0.087079,0.018207,-0.054545,-0.004359,-1,-1,0.6,4.1,-0.0190,0.4,3.0,-0.2,0.1,1.9,1.8,-2.6,-7.3,0.9,1.9,-0.9,3.1,1.3,-0.6,-0.375375,2,1
3,Xavier,BE,38,24,115.6,97.9,0.8713,51.9,51.6,18.4,17.5,34.5,25.5,41.1,33.4,52.0,52.5,34.5,33.4,68.4,1.6,1,63.157895,BE,34,28,117.0,94.3,0.9229,51.4,47.8,17.4,19.3,35.2,26.0,41.7,33.6,49.8,48.6,36.3,31.1,72.4,7.7,0,82.352941,0.105263,-0.166667,-0.012111,0.036772,-0.059222,0.009634,0.073643,0.054348,-0.102857,-0.020290,-0.019608,-0.014599,-0.005988,0.042308,0.074286,-0.052174,0.068862,-0.058480,-3.812500,-0.303922,4,-4,-1.4,3.6,-0.0516,0.5,3.8,1.0,-1.8,-0.7,-0.5,-0.6,-0.2,2.2,3.9,-1.8,2.3,-4.0,-6.1,-19.195046,1,0
4,Florida,SEC,36,27,116.6,88.9,0.9576,51.6,45.8,16.9,21.0,32.1,29.5,40.2,34.0,50.3,45.7,35.9,30.5,70.2,6.4,1,75.000000,SEC,36,21,110.0,93.2,0.8703,48.5,46.7,16.0,18.6,32.9,29.5,39.5,35.7,48.9,44.4,31.9,34.3,69.9,0.1,0,58.333333,0.000000,0.222222,0.056604,-0.048369,0.091165,0.060078,-0.019651,0.053254,0.114286,-0.024922,0.000000,0.017413,-0.050000,0.027833,0.028446,0.111421,-0.124590,0.004274,0.984375,0.222222,0,6,6.6,-4.3,0.0873,3.1,-0.9,0.9,2.4,-0.8,0.0,0.7,-1.7,1.4,1.3,4.0,-3.8,0.3,6.3,16.666667,1,0


In [15]:
x_train, y_train = training_data.drop(["TEAM","POSTSEASON"],axis = 1), training_data["POSTSEASON"]
x_test, y_test = test_data.drop(["TEAM","POSTSEASON"],axis = 1),test_data["POSTSEASON"]

In [16]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((351, 85), (351,), (351, 85), (351,))

In [17]:
from sklearn.preprocessing import LabelEncoder
x_train = pd.get_dummies(x_train, columns=["CONF_last","CONF_llast"])
x_test = pd.get_dummies(x_test, columns=["CONF_last","CONF_llast"])

In [18]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((351, 147), (351,), (351, 147), (351,))

In [19]:
x_train[x_train.isin([np.nan, np.inf, -np.inf]).any(1)].shape, np.where(x_train.values >= np.finfo(np.float64).max), x_test[x_test.isin([np.nan, np.inf, -np.inf]).any(1)].shape, np.where(x_test.values >= np.finfo(np.float64).max)
x_train.replace([np.nan, np.inf, -np.inf], 0,inplace = True)
x_test.replace([np.nan, np.inf, -np.inf], 0,inplace = True)
x_train[x_train.isin([np.nan, np.inf, -np.inf]).any(1)].shape, np.where(x_train.values >= np.finfo(np.float64).max), x_test[x_test.isin([np.nan, np.inf, -np.inf]).any(1)].shape, np.where(x_test.values >= np.finfo(np.float64).max)

((0, 147),
 (array([], dtype=int64), array([], dtype=int64)),
 (0, 147),
 (array([], dtype=int64), array([], dtype=int64)))

In [20]:
x_train.head()

,G_last,W_last,ADJOE_last,ADJDE_last,BARTHAG_last,EFG_O_last,EFG_D_last,TOR_last,TORD_last,ORB_last,DRB_last,FTR_last,FTRD_last,2P_O_last,2P_D_last,3P_O_last,3P_D_last,ADJ_T_last,WAB_last,POSTSEASON_last,WINPERCENT_last,G_llast,W_llast,ADJOE_llast,ADJDE_llast,BARTHAG_llast,EFG_O_llast,EFG_D_llast,TOR_llast,TORD_llast,ORB_llast,DRB_llast,FTR_llast,FTRD_llast,2P_O_llast,2P_D_llast,3P_O_llast,3P_D_llast,ADJ_T_llast,WAB_llast,POSTSEASON_llast,WINPERCENT_llast,G_delta,W_delta,ADJOE_delta,ADJDE_delta,BARTHAG_delta,EFG_O_delta,EFG_D_delta,TOR_delta,TORD_delta,ORB_delta,DRB_delta,FTR_delta,FTRD_delta,2P_O_delta,2P_D_delta,3P_O_delta,3P_D_delta,ADJ_T_delta,WAB_delta,WINPERCENT_delta,G_diff,W_diff,ADJOE_diff,ADJDE_diff,BARTHAG_diff,EFG_O_diff,EFG_D_diff,TOR_diff,TORD_diff,ORB_diff,DRB_diff,FTR_diff,FTRD_diff,2P_O_diff,2P_D_diff,3P_O_diff,3P_D_diff,ADJ_T_diff,WAB_diff,WINPERCENT_diff,win_count,CONF_last_A10,CONF_last_ACC,CONF_last_AE,CONF_last_ASun,CONF_last_Amer,CONF_last_B10,CONF_last_B12,CONF_last_BE,CONF_last_BSky,CONF_last_BSth,CONF_last_BW,CONF_last_CAA,CONF_last_CUSA,CONF_last_Horz,CONF_last_Ivy,CONF_last_MAAC,CONF_last_MAC,CONF_last_MEAC,CONF_last_MVC,CONF_last_MWC,CONF_last_NEC,CONF_last_OVC,CONF_last_P12,CONF_last_Pat,CONF_last_SB,CONF_last_SC,CONF_last_SEC,CONF_last_SWAC,CONF_last_Slnd,CONF_last_Sum,CONF_last_WAC,CONF_last_WCC,CONF_llast_A10,CONF_llast_ACC,CONF_llast_AE,CONF_llast_ASun,CONF_llast_Amer,CONF_llast_B10,CONF_llast_B12,CONF_llast_BE,CONF_llast_BSky,CONF_llast_BSth,CONF_llast_BW,CONF_llast_CAA,CONF_llast_CUSA,CONF_llast_Horz,CONF_llast_Ivy,CONF_llast_MAAC,CONF_llast_MAC,CONF_llast_MEAC,CONF_llast_MVC,CONF_llast_MWC,CONF_llast_NEC,CONF_llast_OVC,CONF_llast_P12,CONF_llast_Pat,CONF_llast_SB,CONF_llast_SC,CONF_llast_SEC,CONF_llast_SWAC,CONF_llast_Slnd,CONF_llast_Sum,CONF_llast_WAC,CONF_llast_WCC
0,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,17.1,30.0,26.2,39.0,26.9,56.3,40.0,38.2,29.0,71.5,7.7,1,94.871795,35,27,117.4,94.5,0.9238,55.2,44.8,17.1,15.1,32.1,26.0,34.4,28.1,54.3,44.4,37.8,30.3,68.2,2.1,0,77.142857,0.102564,0.270270,0.003396,-0.095017,0.050370,0.024735,-0.090024,-0.055556,0.116959,-0.070000,0.007634,0.117949,-0.044610,0.035524,-0.110000,0.010471,-0.044828,0.046154,0.727273,0.186873,4,10,0.4,-8.2,0.0490,1.4,-3.7,-0.9,2.0,-2.1,0.2,4.6,-1.2,2.0,-4.4,0.4,-1.3,3.3,5.6,17.728938,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,39,33,121.0,91.5,0.9615,51.7,48.1,16.2,18.6,41.3,25.0,34.3,31.6,51.0,46.3,35.5,33.9,72.8,8.4,1,84.615385,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,18.2,40.7,30.0,32.3,30.4,53.9,44.6,32.7,36.2,71.7,8.6,1,82.500000,-0.025641,0.000000,-0.019008,-0.037158,0.008736,-0.017408,0.000000,0.049383,0.021505,0.014528,-0.200000,0.058309,0.037975,-0.056863,0.036717,0.078873,-0.067847,0.015110,-0.023810,0.025000,-1,0,-2.3,-3.4,0.0084,-0.9,0.0,0.8,0.4,0.6,-5.0,2.0,1.2,-2.9,1.7,2.8,-2.3,1.1,-0.2,2.115385,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,36,31,121.5,94.5,0.9472,56.1,48.1,17.6,18.6,34.1,29.7,36.0,30.0,53.6,45.3,40.4,35.6,71.4,11.0,1,86.111111,37,32,120.9,90.4,0.9662,55.7,45.1,17.8,18.5,32.2,27.9,38.6,37.3,52.7,43.4,41.3,32.5,70.1,11.6,1,86.486486,-0.027778,-0.032258,0.004938,0.043386,-0.020059,0.007130,0.062370,-0.011364,0.005376,0.055718,0.060606,-0.072222,-0.243333,0.016791,0.041943,-0.022277,0.087079,0.018207,-0.054545,-0.004359,-1,-1,0.6,4.1,-0.0190,0.4,3.0,-0.2,0.1,1.9,1.8,-2.6,-7.3,0.9,1.9,-0.9,3.1,1.3,-0.6,-0.375375,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,38,24,115.6,97.9,0.8713,51.9,51.6,18.4,17.5,34.5,25.5,41.1,33.4,52.0,52.5,34.5,33.4,68.4,1.6,1,63.157895,34,28,117.0,94.3,0.9229,51.4,47.8,17.4,19.3,35.2,26.0,41.7,33.6,49.8,48.6,36.3,31.1,72.4,7.7,0,82.352941,0.105263,-0.166667,-0.012111,0.036772,-0.059222,0.009634,0.073643,0.054348,-0.102857

In [21]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(random_state = 42)
lr_model.fit(x_train,y_train)

lr_prediction = lr_model.predict(x_test)

from sklearn.metrics import confusion_matrix, classification_report
labels = ["Negative","Positive"]
print(classification_report(y_test,lr_prediction))
pd.DataFrame(confusion_matrix(y_test,lr_prediction), index = labels, columns = labels)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       343
           1       0.20      0.25      0.22         8

    accuracy                           0.96       351
   macro avg       0.59      0.61      0.60       351
weighted avg       0.96      0.96      0.96       351



C:\Users\shuba\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Negative,Positive
Negative,335,8
Positive,6,2


In [28]:
from feature_selection import get_imp_features
selected_feature = get_imp_features(x_train,y_train, no_of_features = 25)

Fitting estimator with 26 features.


In [29]:
imp_features = selected_feature.loc[selected_feature['Total']>3].Feature

In [30]:
imp_features

1     WAB_last     
2     2P_O_last    
3     WAB_llast    
4     TOR_diff     
5     TORD_last    
6     ORB_llast    
7     ADJOE_last   
8     ADJDE_last   
9     2P_O_diff    
10    FTR_last     
11    EFG_O_last   
12    DRB_llast    
13    BARTHAG_llast
14    BARTHAG_last 
15    ADJDE_llast  
16    3P_D_last    
Name: Feature, dtype: object

In [31]:
x_train = x_train[imp_features]
x_test = x_test[imp_features]

In [58]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(random_state = 42)
lr_model.fit(x_train,y_train)

lr_prediction = lr_model.predict(x_test)

from sklearn.metrics import confusion_matrix, classification_report
labels = ["Negative","Positive"]
print(classification_report(y_test,lr_prediction))
pd.DataFrame(confusion_matrix(y_test,lr_prediction), index = labels, columns = labels)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       343
           1       0.38      0.38      0.38         8

    accuracy                           0.97       351
   macro avg       0.68      0.68      0.68       351
weighted avg       0.97      0.97      0.97       351



,Negative,Positive
Negative,338,5
Positive,5,3


In [59]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 42)
o_x_train, o_y_train = smote.fit_resample(x_train,y_train)

from sklearn.linear_model import LogisticRegression

o_lr_model = LogisticRegression(random_state = 42)
o_lr_model.fit(o_x_train,o_y_train)

o_lr_prediction = o_lr_model.predict(x_test)

from sklearn.metrics import confusion_matrix, classification_report
labels = ["Negative","Positive"]
print(classification_report(y_test,o_lr_prediction))
pd.DataFrame(confusion_matrix(y_test,o_lr_prediction), index = labels, columns = labels)

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       343
           1       0.29      0.62      0.40         8

    accuracy                           0.96       351
   macro avg       0.64      0.80      0.69       351
weighted avg       0.98      0.96      0.96       351



,Negative,Positive
Negative,331,12
Positive,3,5


In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

g_lr_model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01, 0.05]

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)

gs_lr_model = GridSearchCV(g_lr_model, grid, cv = 2, verbose = 1, n_jobs = -1)
gs_lr_model.fit(o_x_train,o_y_train)

print(gs_lr_model.best_params_)

gs_lr_best_model = gs_lr_model.best_estimator_
gs_lr_prediction = gs_lr_best_model.predict(x_test)

from sklearn.metrics import confusion_matrix, classification_report
labels = ["Negative","Positive"]
print(classification_report(y_test,gs_lr_prediction))
pd.DataFrame(confusion_matrix(y_test,gs_lr_prediction), index = labels, columns = labels)

Fitting 2 folds for each of 18 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  36 | elapsed:    0.6s remaining:    0.4s


{'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       343
           1       0.31      0.62      0.42         8

    accuracy                           0.96       351
   macro avg       0.65      0.80      0.70       351
weighted avg       0.98      0.96      0.97       351



[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    1.1s finished


,Negative,Positive
Negative,332,11
Positive,3,5


In [61]:
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

rc_model = RidgeClassifier()
alpha = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# define grid search
grid = dict(alpha=alpha)

gs_rc__model = GridSearchCV(rc_model, grid, cv = 2, verbose = 1, n_jobs = -1)
gs_rc__model.fit(o_x_train,o_y_train)

print(gs_rc__model.best_params_)

gs_rc_best_model = gs_rc__model.best_estimator_
gs_rc_prediction = gs_rc_best_model.predict(x_test)

from sklearn.metrics import confusion_matrix, classification_report
labels = ["Negative","Positive"]
print(classification_report(y_test,gs_rc_prediction))
pd.DataFrame(confusion_matrix(y_test,gs_rc_prediction), index = labels, columns = labels)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
{'alpha': 0.1}
              precision    recall  f1-score   support

           0       0.99      0.90      0.95       343
           1       0.15      0.75      0.26         8

    accuracy                           0.90       351
   macro avg       0.57      0.83      0.60       351
weighted avg       0.97      0.90      0.93       351



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.1s finished


,Negative,Positive
Negative,310,33
Positive,2,6


In [62]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold

knn_model = KNeighborsClassifier()
n_neighbors = range(1, 21, 2)
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']
# define grid search
grid = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
gs_kn_model = GridSearchCV(knn_model, grid, cv=cv, verbose = 1, n_jobs = -1)
gs_kn_model.fit(o_x_train,o_y_train)

print(gs_kn_model.best_params_)

gs_knn_best_model = gs_kn_model.best_estimator_
gs_knn_prediction = gs_knn_best_model.predict(x_test)

from sklearn.metrics import confusion_matrix, classification_report
labels = ["Negative","Positive"]
print(classification_report(y_test,gs_knn_prediction))
pd.DataFrame(confusion_matrix(y_test,gs_knn_prediction), index = labels, columns = labels)

Fitting 30 folds for each of 60 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 1200 tasks      | elapsed:    5.8s


{'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}
              precision    recall  f1-score   support

           0       0.99      0.93      0.96       343
           1       0.15      0.50      0.23         8

    accuracy                           0.92       351
   macro avg       0.57      0.72      0.59       351
weighted avg       0.97      0.92      0.94       351



[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed:    8.3s finished


,Negative,Positive
Negative,320,23
Positive,4,4


In [71]:
from sklearn.svm import SVC

sv_model = SVC()
kernel = ['poly', 'rbf', 'sigmoid']
C = [50, 10, 1.0, 0.1, 0.01]
gamma = ['scale']
# define grid search
grid = dict(kernel=kernel,C=C,gamma=gamma)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
gs_sv_model = GridSearchCV(sv_model, grid, cv=2, verbose = 1, n_jobs = -1)
gs_sv_model.fit(o_x_train,o_y_train)

print(gs_sv_model.best_params_)

gs_sv_best_model = gs_sv_model.best_estimator_
gs_sv_prediction = gs_sv_best_model.predict(x_test)

from sklearn.metrics import confusion_matrix, classification_report
labels = ["Negative","Positive"]
print(classification_report(y_test,gs_sv_prediction))
pd.DataFrame(confusion_matrix(y_test,gs_sv_prediction), index = labels, columns = labels)

Fitting 2 folds for each of 15 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 50, 'gamma': 'scale', 'kernel': 'poly'}
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       343
           1       0.18      0.50      0.27         8

    accuracy                           0.94       351
   macro avg       0.58      0.72      0.62       351
weighted avg       0.97      0.94      0.95       351



[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    9.3s finished


,Negative,Positive
Negative,325,18
Positive,4,4


In [72]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state = 42)
rf_model.fit(o_x_train,o_y_train)

rf_prediction = rf_model.predict(x_test) 

from sklearn.metrics import confusion_matrix, classification_report
labels = ["Negative","Positive"]
print(classification_report(y_test,rf_prediction))
pd.DataFrame(confusion_matrix(y_test,rf_prediction), index = labels, columns = labels)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       343
           1       0.22      0.25      0.24         8

    accuracy                           0.96       351
   macro avg       0.60      0.61      0.61       351
weighted avg       0.97      0.96      0.96       351



,Negative,Positive
Negative,336,7
Positive,6,2


In [73]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

min_samples_split = [2, 5, 10, 15]

min_samples_leaf = [1, 2, 4, 6]

bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

gs_rf_model = RandomizedSearchCV(RandomForestClassifier(), random_grid, cv = 2, verbose = 1, n_jobs = -1)
gs_rf_model.fit(o_x_train,o_y_train)

print(gs_rf_model.best_params_)

gs_rf_best_model = gs_rf_model.best_estimator_
gs_rf_prediction = gs_rf_best_model.predict(x_test)

from sklearn.metrics import confusion_matrix, classification_report
labels = ["Negative","Positive"]
print(classification_report(y_test,gs_rf_prediction))
pd.DataFrame(confusion_matrix(y_test,gs_rf_prediction), index = labels, columns = labels)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10, 15], 'min_samples_leaf': [1, 2, 4, 6], 'bootstrap': [True, False]}
Fitting 2 folds for each of 10 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   17.0s finished


{'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 110, 'bootstrap': False}
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       343
           1       0.22      0.25      0.24         8

    accuracy                           0.96       351
   macro avg       0.60      0.61      0.61       351
weighted avg       0.97      0.96      0.96       351



,Negative,Positive
Negative,336,7
Positive,6,2


# Save Model

In [74]:
import joblib
joblib.dump(lr_model, "lr_model.rb")
joblib.dump(o_lr_model, "o_lr_model.rb")
joblib.dump(gs_lr_best_model, "gs_lr_best_model.rb")
joblib.dump(gs_rc_best_model, "gs_rc_best_model.rb")
joblib.dump(gs_knn_best_model, "gs_knn_best_model.rb")
joblib.dump(gs_sv_best_model, "gs_sv_best_model.rb")
joblib.dump(rf_model, "rf_model.rb")
joblib.dump(gs_rf_best_model, "gs_rf_best_model.rb")

['gs_rf_best_model.rb']

In [75]:
#Copy model to other variable
final_best_model = gs_lr_best_model

check_prediction = final_best_model.predict(x_test)

from sklearn.metrics import confusion_matrix, classification_report
labels = ["Negative","Positive"]
print(classification_report(y_test,check_prediction))
pd.DataFrame(confusion_matrix(y_test,check_prediction), index = labels, columns = labels)

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       343
           1       0.31      0.62      0.42         8

    accuracy                           0.96       351
   macro avg       0.65      0.80      0.70       351
weighted avg       0.98      0.96      0.97       351



,Negative,Positive
Negative,332,11
Positive,3,5


In [76]:
##########################
#Training_data Preparation
##########################

llast_year = year_list[1]
last_year = year_list[2]

data_llast = df[df.YEAR == llast_year]
data_llast.drop("YEAR",inplace = True, axis = 1)
col_list = data_llast.columns.tolist()
data_llast.columns = [str(col) + '_'+ 'llast' if col != "TEAM" else str(col) for col in data_llast.columns]

data_last = df[df.YEAR == last_year]
data_last.drop(["YEAR"],inplace = True, axis = 1)
data_last.columns = [str(col) + '_'+ 'last'  if col != "TEAM" else str(col)  for col in data_last.columns]

training_data = pd.merge(data_last, data_llast, on="TEAM")

for col in col_list[2:]:
    training_data[col + "_delta"] = (training_data[col + "_" + 'last'] - training_data[col + "_" + 'llast'])/training_data[col + "_" + 'last']

for col in col_list[2:]:
    training_data[col + "_diff"] = (training_data[col + "_" + 'last'] - training_data[col + "_" + 'llast'])
    
training_data["win_count"] = training_data["POSTSEASON_llast"] + training_data["POSTSEASON_last"]
    
training_data.drop(["POSTSEASON_delta"],axis = 1,inplace = True)
training_data.drop(["POSTSEASON_diff"],axis = 1,inplace = True)

training_data.isnull().sum()

TEAM                0
CONF_last           0
G_last              0
W_last              0
ADJOE_last          0
ADJDE_last          0
BARTHAG_last        0
EFG_O_last          0
EFG_D_last          0
TOR_last            0
TORD_last           0
ORB_last            0
DRB_last            0
FTR_last            0
FTRD_last           0
2P_O_last           0
2P_D_last           0
3P_O_last           0
3P_D_last           0
ADJ_T_last          0
WAB_last            0
POSTSEASON_last     0
WINPERCENT_last     0
CONF_llast          0
G_llast             0
W_llast             0
ADJOE_llast         0
ADJDE_llast         0
BARTHAG_llast       0
EFG_O_llast         0
EFG_D_llast         0
TOR_llast           0
TORD_llast          0
ORB_llast           0
DRB_llast           0
FTR_llast           0
FTRD_llast          0
2P_O_llast          0
2P_D_llast          0
3P_O_llast          0
3P_D_llast          0
ADJ_T_llast         0
WAB_llast           0
POSTSEASON_llast    0
WINPERCENT_llast    0
G_delta   

In [77]:
tlabel_year = year_list[3]
train_label_data = df[df.YEAR == tlabel_year]
train_label_data = train_label_data[["TEAM","POSTSEASON"]]
training_data = pd.merge(training_data, train_label_data, on="TEAM")

In [78]:
##########################
#Test_data Preparation
##########################
llast_year = year_list[2]
last_year = year_list[3]

data_llast = df[df.YEAR == llast_year]
data_llast.drop("YEAR",inplace = True, axis = 1)
col_list = data_llast.columns.tolist()
data_llast.columns = [str(col) + '_'+ 'llast' if col != "TEAM" else str(col) for col in data_llast.columns]

data_last = df[df.YEAR == last_year]
data_last.drop(["YEAR"],inplace = True, axis = 1)
data_last.columns = [str(col) + '_'+ 'last'  if col != "TEAM" else str(col)  for col in data_last.columns]

test_data = pd.merge(data_last, data_llast, on="TEAM")

for col in col_list[2:]:
    test_data[col + "_delta"] = (test_data[col + "_" + 'last'] - test_data[col + "_" + 'llast'])/test_data[col + "_" + 'last']

for col in col_list[2:]:
    test_data[col + "_diff"] = (test_data[col + "_" + 'last'] - test_data[col + "_" + 'llast'])

test_data["win_count"] = test_data["POSTSEASON_llast"] + test_data["POSTSEASON_last"]
    
test_data.drop(["POSTSEASON_delta"],axis = 1,inplace = True)
test_data.drop(["POSTSEASON_diff"],axis = 1,inplace = True)

test_data.isnull().sum()

TEAM                0
CONF_last           0
G_last              0
W_last              0
ADJOE_last          0
ADJDE_last          0
BARTHAG_last        0
EFG_O_last          0
EFG_D_last          0
TOR_last            0
TORD_last           0
ORB_last            0
DRB_last            0
FTR_last            0
FTRD_last           0
2P_O_last           0
2P_D_last           0
3P_O_last           0
3P_D_last           0
ADJ_T_last          0
WAB_last            0
POSTSEASON_last     0
WINPERCENT_last     0
CONF_llast          0
G_llast             0
W_llast             0
ADJOE_llast         0
ADJDE_llast         0
BARTHAG_llast       0
EFG_O_llast         0
EFG_D_llast         0
TOR_llast           0
TORD_llast          0
ORB_llast           0
DRB_llast           0
FTR_llast           0
FTRD_llast          0
2P_O_llast          0
2P_D_llast          0
3P_O_llast          0
3P_D_llast          0
ADJ_T_llast         0
WAB_llast           0
POSTSEASON_llast    0
WINPERCENT_llast    0
G_delta   

In [79]:
test_data["POSTSEASON"] = 0

In [80]:
x_train, y_train = training_data.drop(["TEAM","POSTSEASON"],axis = 1), training_data["POSTSEASON"]
x_test, y_test = test_data.drop(["TEAM","POSTSEASON"],axis = 1),test_data["POSTSEASON"]
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((351, 85), (351,), (351, 85), (351,))

In [81]:
from sklearn.preprocessing import LabelEncoder
x_train = pd.get_dummies(x_train, columns=["CONF_last","CONF_llast"])
x_test = pd.get_dummies(x_test, columns=["CONF_last","CONF_llast"])
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((351, 147), (351,), (351, 147), (351,))

In [82]:
x_train[x_train.isin([np.nan, np.inf, -np.inf]).any(1)].shape, np.where(x_train.values >= np.finfo(np.float64).max), x_test[x_test.isin([np.nan, np.inf, -np.inf]).any(1)].shape, np.where(x_test.values >= np.finfo(np.float64).max)
x_train.replace([np.nan, np.inf, -np.inf], 0,inplace = True)
x_test.replace([np.nan, np.inf, -np.inf], 0,inplace = True)
x_train[x_train.isin([np.nan, np.inf, -np.inf]).any(1)].shape, np.where(x_train.values >= np.finfo(np.float64).max), x_test[x_test.isin([np.nan, np.inf, -np.inf]).any(1)].shape, np.where(x_test.values >= np.finfo(np.float64).max)

((0, 147),
 (array([], dtype=int64), array([], dtype=int64)),
 (0, 147),
 (array([], dtype=int64), array([], dtype=int64)))

In [83]:
x_train = x_train[imp_features]
x_test = x_test[imp_features]

In [86]:
x_test.head()

,WAB_last,2P_O_last,WAB_llast,TOR_diff,TORD_last,ORB_llast,ADJOE_last,ADJDE_last,2P_O_diff,FTR_last,EFG_O_last,DRB_llast,BARTHAG_llast,BARTHAG_last,ADJDE_llast,3P_D_last
0,7.0,52.8,4.7,-0.1,22.8,32.9,115.2,85.2,1.7,32.9,53.5,28.7,0.9170,0.9696,91.5,29.7
1,11.1,52.5,11.3,0.7,17.5,27.0,123.0,89.9,2.4,29.1,55.2,25.6,0.9612,0.9736,86.7,28.9
2,11.2,58.0,6.6,0.0,19.4,38.8,118.9,89.2,2.0,33.2,53.6,30.0,0.9592,0.9646,92.9,29.9
3,6.1,51.5,7.6,0.0,18.6,28.0,122.8,94.3,-2.6,29.9,53.6,28.0,0.9484,0.9539,95.7,34.2
4,8.8,52.9,4.3,0.0,17.9,35.7,117.5,89.8,1.7,41.9,53.0,31.0,0.9000,0.9568,95.6,34.3


# Final Prediction for Year 2020 - Team likely to be qualified

In [171]:
final_best_model = gs_lr_best_model
final_prediction = final_best_model.predict(x_test)
final_prediction_prob = final_best_model.predict_proba(x_test)
predicted_probability = final_prediction_prob[:, 1]

In [172]:
output = pd.DataFrame({"team" : test_data.TEAM,"prediction":final_prediction, "class_not_qualified" : final_prediction_prob[:, 0], "class_qualified" : final_prediction_prob[:, 1] })

## Team likely to be qualified

In [173]:
output[output.prediction == 1]

,team,prediction,class_not_qualified,class_qualified
0,Texas Tech,1,0.0115373832,0.9884626168
1,Virginia,1,0.0000843891,0.9999156109
2,Duke,1,0.2229032342,0.7770967658
4,Kentucky,1,0.0000224562,0.9999775438
26,Pittsburgh,1,0.2383182347,0.7616817653
53,Nebraska,1,0.0365443564,0.9634556436
291,UCF,1,0.0000036283,0.9999963717
297,Kansas,1,0.0009367878,0.9990632122
304,Washington,1,0.0025045789,0.9974954211
315,Kansas St.,1,0.0000290805,0.9999709195


## Top 8 team to be qualified based on probability score

In [174]:
index = (output.class_qualified).argsort()[-8:][::-1]
output.iloc[index]

,team,prediction,class_not_qualified,class_qualified
350,Tennessee,1,0.0000000000,1.0000000000
326,Nevada,1,0.0000018648,0.9999981352
291,UCF,1,0.0000036283,0.9999963717
4,Kentucky,1,0.0000224562,0.9999775438
315,Kansas St.,1,0.0000290805,0.9999709195
1,Virginia,1,0.0000843891,0.9999156109
340,St. John's,1,0.0003778009,0.9996221991
297,Kansas,1,0.0009367878,0.9990632122


In [177]:
output.iloc[index].team.tolist()

['Tennessee',
 'Nevada',
 'UCF',
 'Kentucky',
 'Kansas St.',
 'Virginia',
 "St. John's",
 'Kansas']